In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_8.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_6.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_31.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_22.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_3.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_45.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_37.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_disease/xrays/val_19.png
/kaggle/input/dentex-challenge-2023/validation_data/validation_data/quadrant_enumeration_di

In [3]:
!nvidia-smi

Mon Dec 29 19:53:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             33W /  250W |    3329MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
# ==============================================================================
# 1. SETUP & DEPENDENCIES
# ==============================================================================
import os
import sys
import subprocess

# Quick GPU Check before installing anything
print("🔍 Checking System Hardware...")
try:
    gpu_info = subprocess.check_output(['nvidia-smi'], encoding='utf-8')
    print("✅ Hardware Check Passed: GPU found.")
    print(gpu_info.split('\n')[2]) # Print GPU Name
except:
    print("❌ CRITICAL WARNING: No GPU detected via nvidia-smi.")
    print("   Please go to Settings -> Accelerator -> GPU P100/T4 on the right panel.")

# Install Libraries
print("\n📦 Installing dependencies (Ultralytics + Lightly)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics", "lightly", "-q"])

import torch
import torch.nn as nn
from ultralytics import YOLO
from lightly.data import LightlyDataset
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.loss import NTXentLoss
from lightly.models.modules import SimCLRProjectionHead
from tqdm import tqdm

# ==============================================================================
# 2. CONFIGURATION
# ==============================================================================

# DEVICE ROUTING: This variable controls where the computation happens
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# PATHS (Adjust these for your specific environment)
# In Kaggle, input data usually lives in /kaggle/input/
UNLABELED_DIR = "/kaggle/input/dentex-challenge-2023/training_data/training_data/unlabelled/xrays" 
OUTPUT_FILENAME = "yolov8n_dentex_ssl.pt"

# HYPERPARAMETERS
EPOCHS = 50
BATCH_SIZE = 64  # T4 can usually handle 128 for ResNet/YOLO backbones
LEARNING_RATE = 0.01

print(f"\n⚙️ CONFIGURATION:")
print(f"   Device: {DEVICE} (Strict routing enabled)")
print(f"   Data Dir: {UNLABELED_DIR}")
print(f"   Batch Size: {BATCH_SIZE}")

# ==============================================================================
# 3. MODEL DEFINITION
# ==============================================================================

class YoloBackboneSSL(nn.Module):
    def __init__(self, yolo_model):
        super().__init__()
        # Extract CSP-Darknet Backbone (Layers 0-9)
        self.backbone = yolo_model.model.model[:10]
        
        # SimCLR Projection Head
        # Maps 256-dim backbone features to 128-dim embedding space
        self.projection_head = SimCLRProjectionHead(input_dim=256, hidden_dim=256, output_dim=128)

    def forward(self, x):
        # 1. Pass through Backbone
        features = self.backbone(x)
        
        # YOLO backbone might return a list [P3, P4, P5], we want the last one
        if isinstance(features, list):
            features = features[-1]
            
        # 2. Global Average Pooling (Spatial -> Vector)
        # Input: [Batch, 256, H, W] -> Output: [Batch, 256]
        features = nn.functional.adaptive_avg_pool2d(features, (1, 1)).flatten(start_dim=1)
        
        # 3. Project to Embedding
        return self.projection_head(features)

# ==============================================================================
# 4. TRAINING LOOP
# ==============================================================================

def train_and_export():
    if str(DEVICE) == "cpu":
        user_input = input("⚠️ WARNING: You are running on CPU. This will be incredibly slow. Continue? (y/n): ")
        if user_input.lower() != 'y':
            return

    print("\n🚀 Initializing Model...")
    # Load standard YOLOv8n to get the architecture
    base_model = YOLO('yolov8n.pt')
    
    # Wrap it for SSL and ROUTE TO GPU
    ssl_model = YoloBackboneSSL(base_model).to(DEVICE)
    print(f"   Model moved to: {next(ssl_model.parameters()).device}")

    # Data Transforms (SimCLR specific)
    transform = SimCLRTransform(input_size=512, vf_prob=0.0) # No vertical flip for teeth
    
    # Dataset & Loader
    if not os.path.exists(UNLABELED_DIR):
        raise FileNotFoundError(f"Data directory not found: {UNLABELED_DIR}")
        
    dataset = LightlyDataset(input_dir=UNLABELED_DIR, transform=transform)
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=2, 
        drop_last=True,
        pin_memory=True # Faster data transfer to GPU
    )
    
    print(f"   Loaded {len(dataset)} images.")

    # Optimizer & Loss
    criterion = NTXentLoss()
    optimizer = torch.optim.SGD(ssl_model.parameters(), lr=LEARNING_RATE, momentum=0.9)

    print("\n🧠 Starting SimCLR Training...")
    ssl_model.train()
    
    for epoch in range(EPOCHS):
        total_loss = 0
        
        # Progress bar
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
        
        for (x0, x1), _, _ in pbar:
            # === CRITICAL: DATA TO GPU ROUTING ===
            x0 = x0.to(DEVICE)
            x1 = x1.to(DEVICE)
            
            # Forward pass
            z0 = ssl_model(x0)
            z1 = ssl_model(x1)
            
            # Loss calculation
            loss = criterion(z0, z1)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Optional: Update progress bar description with current loss
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} - Avg Loss: {avg_loss:.4f}")

    # ==========================================================================
    # 5. EXPORT & TRANSPLANT WEIGHTS
    # ==========================================================================
    print("\n💾 Saving Backbone Weights...")
    torch.save(ssl_model.backbone.state_dict(), "temp_backbone.pth")
    
    print("🔄 Transplanting weights to YOLOv8 Detector format...")
    # Load a fresh detection model
    final_model = YOLO('yolov8n.pt')
    full_state_dict = final_model.model.state_dict()
    ssl_state_dict = torch.load("temp_backbone.pth")
    
    transferred_count = 0
    for key, val in ssl_state_dict.items():
        # Clean the key name (remove 'backbone.' prefix if present)
        clean_key = key.replace("backbone.", "")
        
        # Map to YOLO format (model.0.conv...)
        target_key = f"model.{clean_key}"
        
        # Check if key exists and shapes match
        if target_key in full_state_dict:
            if full_state_dict[target_key].shape == val.shape:
                full_state_dict[target_key] = val
                transferred_count += 1
    
    # Load modified weights into model
    final_model.model.load_state_dict(full_state_dict)
    
    # Save final PT file
    final_model.save(OUTPUT_FILENAME)
    
    print(f"\n🎉 SUCCESS! Transferred {transferred_count} layers.")
    print(f"✅ Final model saved as: {OUTPUT_FILENAME}")
    print(f"👉 Please download '{OUTPUT_FILENAME}' from the Output tab.")

if __name__ == "__main__":
    train_and_export()

🔍 Checking System Hardware...
✅ Hardware Check Passed: GPU found.
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |

📦 Installing dependencies (Ultralytics + Lightly)...

⚙️ CONFIGURATION:
   Device: cuda (Strict routing enabled)
   Data Dir: /kaggle/input/dentex-challenge-2023/training_data/training_data/unlabelled/xrays
   Batch Size: 64

🚀 Initializing Model...
   Model moved to: cuda:0
   Loaded 1571 images.

🧠 Starting SimCLR Training...


Epoch 1 - Avg Loss: 4.6730


Epoch 2 - Avg Loss: 4.5344


Epoch 3 - Avg Loss: 4.4821


Epoch 4 - Avg Loss: 4.4493


...

Epoch 46 - Avg Loss: 4.1491


Epoch 47 - Avg Loss: 4.1737


Epoch 48 - Avg Loss: 4.1434


Epoch 49 - Avg Loss: 4.1485


Epoch 50 - Avg Loss: 4.1590

💾 Saving Backbone Weights...
🔄 Transplanting weights to YOLOv8 Detector format...

🎉 SUCCESS! Transferred 162 layers.
✅ Final model saved as: yolov8n_dentex_ssl.pt
👉 Please download 'yolov8n_dentex_ssl.pt' from the Output tab.
